In [151]:
#Libraries 
import pandas as pd
import json
from datetime import datetime
import string
from nltk.corpus import stopwords
from nltk import pos_tag
import nltk 

#Load Press Release file

with open('C:/Users/ADMIN/Downloads/ADP_pr.json', 'r',encoding='utf-8') as myfile:
    data=json.load(myfile)
df = pd.DataFrame(data)


newlist=[]
for i in df['date']:
    m="".join(str(i))
    m=m.lstrip()
    m=m.partition('AM')[0]
    m=m.partition('PM')[0]
    m=m.rsplit(' ', 1)[0]
    m=str(m)[0:]
    newdate=(datetime.strptime(m,'%b %d, %Y')) #%b %d,%Y
    newlist.append(newdate)
       

df1=pd.DataFrame(newlist)
df1.columns = ['date']
df1['date'].values
df['date']=df1['date']

#Load Stock File
df2=pd.read_csv('C:/Users/ADMIN/Downloads/ADP_STOCK.csv')
df['date']=df['date'].astype(str)

res=pd.merge(df,df2,on='date', how='outer')
res[' Close/Last']=res[' Close/Last'].str[1:]
res[' Close/Last']=res[' Close/Last'].replace({',': ''}, regex=True)
res[' Open']=res[' Open'].str[1:]
res[' Open']=res[' Open'].replace({',': ''}, regex=True)
res[' High']=res[' High'].str[1:]
res[' High']=res[' High'].replace({',': ''}, regex=True)
res[' Low']=res[' Low'].str[1:]
res[' Low']=res[' Low'].replace({',': ''}, regex=True)
res=res.dropna()
res[' Close/Last']=res[' Close/Last'].astype(float)
res[' Open']=res[' Open'].astype(float)
res[' High']=res[' High'].astype(float)
res[' Low']=res[' Low'].astype(float)
   
#New Column for DIfference of last and opening stock value 
res['diff']=res[' Close/Last']-res[' Open']

#Getting number of words in press release
res['number_of_words'] = res.content.apply(lambda x: len(x.split()))

#Remove Punctuation 
def remove_punctuation(text):
 no_punct = "".join([c for c in text if c not in string.punctuation])
 return no_punct
res['content']=res['content'].apply(lambda x:remove_punctuation(x))

#Getting POS tag 
tagged_item=res['content'].map(pos_tag)

#Getting count of POS tag
def count_tags(title_with_tags):
    tag_count = {}
    for word, tag in title_with_tags:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1
    return(tag_count)


bc1= pd.DataFrame() #Null Dataframe

#Dictionary for POS tag count 
x=tagged_item.map(count_tags)

#Converting Dictionary into Dataframe 
for row in x:
  bc=pd.DataFrame([row],columns=row.keys())
  bc1=bc1.append(bc)


res['pos']=res['diff']>0
bc1['Nature']=res['pos'].values
bc1['words']=res['number_of_words'].values


#Total positive and negative press release
Tr=(bc1['Nature']==True).sum()
Fl=(bc1['Nature']==False).sum()

#Total words in postiive and negative press release
Tr_cnt= bc1.loc[bc1['Nature']== True, 'words'].sum()
Fl_cnt= bc1.loc[bc1['Nature']== False,'words'].sum()

#Nomralization index
True_N=(Tr*Tr_cnt)
False_N=(Fl*Fl_cnt)

#Normalizing both the press releases 
sei=bc1.loc[bc1['Nature']==True]
sei1=(sei.loc[:, ~sei.columns.isin(['Nature', 'words'])])/True_N
sei1['Nat']=sei['Nature']

sei2=bc1.loc[bc1['Nature']==False]
sei3=(sei2.loc[:, ~sei2.columns.isin(['Nature', 'words'])])/False_N
sei3['Nat']=sei2['Nature']


#Keeping only most common POS of the press release
fio=pd.concat([sei1, sei3])
fio=fio[['NN','VB','VBD','JJ','RB','NNP','CD','DT','Nat']]


         
fio.to_csv('C:/Users/ADMIN/Downloads/TextAnalysis.csv', mode='a', header=False)



,NN,VB,VBD,JJ,RB,NNP,CD,DT,Nat
0,0.037955,0.000317,0.000732,0.005567,0.000061,0.011793,0.001343,0.004261,True
0,0.037247,0.000317,0.000769,0.005665,0.000085,0.011402,0.001355,0.004322,True
0,0.021621,0.000269,0.000635,0.003821,0.000134,0.006727,0.000464,0.003577,True
0,0.025845,0.000195,0.000635,0.004200,0.000110,0.007960,0.000696,0.003223,True
0,0.031485,0.000256,0.000745,0.003821,0.000049,0.008558,0.001318,0.003211,True
0,0.014516,0.000183,0.000391,0.002368,0.000012,0.005030,0.000623,0.001807,True
0,0.013734,0.000085,0.000244,0.001929,0.000024,0.004969,0.000342,0.002002,True
0,0.036246,0.000342,0.000781,0.005359,0.000049,0.011171,0.001282,0.004065,True
0,0.011683,0.000122,0.000293,0.001856,0.000012,0.004041,0.000732,0.001526,True
0,0.013771,0.000098,0.000244,0.001917,0.000024,0.004957,0.000354,0.001941,True


In [ ]:
 # bc=pd.DataFrame.from_dict([row,columns=['CC',
'CD',
'DT',
'EX',
'FW',
'IN',
'JJ',
'JJR',
'JJS',
'LS',
'MD',
'NN',
'NNS',
'NNP',
'NNPS',
'PDT',
'POS',
'PRP',
'PRP$',
'RB',
'RBR',
'RBS',
'RP',
'SYM',
'TO',
'UH',
'VB',
'VBD',
'VBG',
'VBN',
'VBP',
'VBZ',
'WDT',
'WP',
'WP$',
'WRB',
],orient='index')